In [ ]:
!pip install tetra_rp

In [ ]:
import asyncio
import base64
import io
from PIL import Image
from tetra_rp import remote, LiveServerless, GpuGroup
from IPython.display import display

In [ ]:
sd_config = LiveServerless(
    gpus=[GpuGroup.AMPERE_80],
    name="example_image_gen_server",
)

print("GPU configuration created successfully!")
print(f"GPU Group: {GpuGroup.AMPERE_80}")
print(f"Server Name: {sd_config.name}")


In [ ]:
@remote(
    resource_config=sd_config,
    dependencies=["diffusers", "transformers", "torch", "accelerate", "safetensors"],
)
def generate_image(
    prompt,
    negative_prompt="",
    width=512,
    height=512,
    num_inference_steps=30,
    guidance_scale=7.5,
):
    """Generate an image using Stable Diffusion."""
    import torch
    from diffusers import StableDiffusionPipeline
    import io
    import base64
    from PIL import Image
    from pathlib import Path
    import os

    # File-based model caching to avoid reloading
    model_path = Path("/tmp/stable_diffusion_model")
    os.makedirs(model_path, exist_ok=True)

    # Load pipeline
    print("Loading Stable Diffusion pipeline...")
    pipeline = StableDiffusionPipeline.from_pretrained(
        "runwayml/stable-diffusion-v1-5",
        torch_dtype=torch.float16,
        cache_dir=str(model_path),
        local_files_only=(model_path / "snapshots").exists(),
    )

    # Move to GPU
    pipeline = pipeline.to("cuda")

    # Generate image
    print(f"Generating image for prompt: '{prompt}'")
    image = pipeline(
        prompt=prompt,
        negative_prompt=negative_prompt,
        width=width,
        height=height,
        num_inference_steps=num_inference_steps,
        guidance_scale=guidance_scale,
    ).images[0]

    # Convert to base64
    buffered = io.BytesIO()
    image.save(buffered, format="PNG")
    img_str = base64.b64encode(buffered.getvalue()).decode()

    return {"image": img_str, "prompt": prompt, "dimensions": f"{width}x{height}"}

print("Remote image generation function defined!")

In [ ]:
def save_image_from_result(result, filename="generated_image.png"):
    """Save image from the result dictionary to a file."""
    img_data = base64.b64decode(result["image"])
    image = Image.open(io.BytesIO(img_data))
    image.save(filename)
    return image

In [ ]:
print("🚀 Starting image generation...")

result = await generate_image(
    prompt="Superman and batman fighting with spiderman",
    negative_prompt="blurry, distorted, low quality, text, watermark",
    width=768,
    height=512,
)

print("✅ Image generation completed!")


In [ ]:
# Save the image
output_file = "superhero_battle.png"
image = save_image_from_result(result, output_file)

print(f"📸 Image saved to {output_file}")
print(f"🎨 Prompt: {result['prompt']}")
print(f"📏 Dimensions: {result['dimensions']}")

# Display the image in the notebook
display(image)

In [ ]:
prompts = [
    {
        "prompt": "A majestic dragon flying over a medieval castle, fantasy art",
        "negative_prompt": "blurry, low quality, text",
        "filename": "dragon_castle.png"
    },
    {
        "prompt": "A futuristic cyberpunk city at night, neon lights, digital art",
        "negative_prompt": "blurry, low quality, text, watermark",
        "filename": "cyberpunk_city.png"
    },
    {
        "prompt": "A peaceful forest with sunlight streaming through trees, nature photography",
        "negative_prompt": "blurry, low quality, dark, gloomy",
        "filename": "peaceful_forest.png"
    }
]

# Generate multiple images
for i, prompt_config in enumerate(prompts):
    print(f"\n🎨 Generating image {i+1}/3: {prompt_config['prompt'][:50]}...")
    
    result = await generate_image(
        prompt=prompt_config["prompt"],
        negative_prompt=prompt_config["negative_prompt"],
        width=512,
        height=512,
        num_inference_steps=25,  # Faster generation
    )
    
    image = save_image_from_result(result, prompt_config["filename"])
    print(f"✅ Saved: {prompt_config['filename']}")
    
    # Display the image
    display(image)

In [ ]:
# Use this cell to generate images with your own custom prompts!

# %%
# Your custom prompt here
custom_prompt = "A cute robot reading a book in a cozy library"
custom_negative = "blurry, low quality, scary, dark"

print(f"🎯 Generating custom image: {custom_prompt}")

custom_result = await generate_image(
    prompt=custom_prompt,
    negative_prompt=custom_negative,
    width=512,
    height=512,
    num_inference_steps=30,
    guidance_scale=8.0,  # Higher guidance for more prompt adherence
)

custom_image = save_image_from_result(custom_result, "custom_image.png")
print("✅ Custom image generated!")
display(custom_image)

In [ ]:
async def batch_generate_images(prompts_list, base_filename="batch_image"):
    """Generate multiple images in batch."""
    results = []
    
    for i, prompt in enumerate(prompts_list):
        print(f"🔄 Processing {i+1}/{len(prompts_list)}: {prompt[:50]}...")
        
        result = await generate_image(
            prompt=prompt,
            negative_prompt="blurry, low quality, text",
            width=512,
            height=512,
            num_inference_steps=20,  # Faster for batch processing
        )
        
        filename = f"{base_filename}_{i+1:02d}.png"
        image = save_image_from_result(result, filename)
        results.append({"result": result, "image": image, "filename": filename})
        
        print(f"✅ Saved: {filename}")
    
    return results

# Example batch generation
batch_prompts = [
    "A space station orbiting Earth",
    "A vintage car on a mountain road",
    "A magical potion brewing in a cauldron"
]

print("🚀 Starting batch generation...")
batch_results = await batch_generate_images(batch_prompts, "example_batch")
print("🎉 Batch generation completed!")

# %% [markdown]
# ## 10. Clean Up and Summary
# 
# Final summary of generated images.


In [ ]:
# Final summary of generated images.

# %%
import os
import glob

# List all generated PNG files
png_files = glob.glob("*.png")
print("📋 Generated Images Summary:")
print("=" * 40)

for i, filename in enumerate(png_files, 1):
    file_size = os.path.getsize(filename) / 1024  # Size in KB
    print(f"{i:2d}. {filename} ({file_size:.1f} KB)")

print(f"\n🎨 Total images generated: {len(png_files)}")
print("✨ All done! Check your local directory for the generated images.")